### Description of the dataset

After running preprocessing.py script we should get a new file in Training_data folder.
It's a serialized bytes file created by pickle representing training data in a suitable form for training
Loading this file in a variable will result in a tuple which first element is a numpy ndarray representing training entries
and the second element is another array representing corresponding labels 

### Imports

If that is ok let's start by importing needed Packages and Libraries.
Keras is a Python library for deep learning that wraps the efficient numerical libraries TensorFlow and Theano.
Keras allows you to quickly and simply design and train neural networks and deep learning models.

I aim to keep this experience, straightforward and easily understandable for beginners in order to use this like a tutorial to take up with Deep Learning
That's why I will try to make clear all stuffs we are going to see.

You will use scikit-learn to evaluate the model using stratified k-fold cross validation. This is a resampling technique that will provide an estimate of the performance of the model. It does this by splitting the data into k-parts and training the model on all parts except one, which is held out as a test set to evaluate the performance of the model.

To use Keras models with scikit-learn, you must use the KerasClassifier wrapper from the SciKeras module. This class takes a function that creates and returns our neural network model. It also takes arguments that it will pass along to the call to fit(), such as the number of epochs and the batch size.

In [1]:
from tensorflow.keras import Sequential # groups a linear stack of layers into a tf.keras.Model.
from tensorflow.keras.layers import Dense # Just your regular densely-connected NN layer.
from scikeras.wrappers import KerasClassifier # Implementation of the scikit-learn classifier API for Keras.
from sklearn.model_selection import cross_val_score # Evaluate a score by cross-validation.
from sklearn.model_selection import StratifiedKFold # Provides train/test indices to split data in train/test sets.
import pickle # To load data from Training_data folder

2023-05-12 00:24:31.984872: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-12 00:24:32.185221: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-12 00:24:32.214519: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib/x86_64-linux-gnu/gazebo-11/plugins:/opt/ros/humble/opt/rviz_ogre_vendor/li

### Load And prepare Data

Ok let's move on and load data for training

In [2]:
# Load data
with open("../../Training_data/stats_summary.data", "rb") as data:
    dataset = pickle.load(data)
# Split dataset into inputs/outputs for training
Inputs = dataset[0]
Outputs = dataset[1]

print(f"{len(Inputs)} Training examples ready to train")

5925 Training examples ready to train


### Creating the model
We are now ready to create your neural network model using Keras.
Let’s start by defining the function that creates your baseline model

The weights are initialized using a small Gaussian random number. The Rectifier activation function is used. The output layer contains a single neuron in order to make predictions. It uses the sigmoid activation function in order to produce a probability output in the range of 0 to 1 that can easily and automatically be converted to crisp class values.

Finally, you will use the logarithmic loss function (binary_crossentropy) during training, the preferred loss function for binary classification problems. The model also uses the efficient Adam optimization algorithm for gradient descent, and accuracy metrics will be collected when the model is trained.

If you are interested in understanding what is going on behind the below codes. 
I did another github implementing a neural net from scratch using just numpy on here: https://github.com/diarray-hub/covid_classification 

I would also recommand you the book neural-networks-and-deep-learning By Michael Nielsen

In [3]:
from typing import Union # To type variables that accept more than one value types (Not needed in Python3.10 or newer)

def Create_Sequential(layers : list[int], activations : Union[str, list[str]] = "sigmoid", 
                      optimizer="adam", metrics=['accuracy'], loss="binary_crossentropy"):
    """
    We will use this function to create our models

    Args:
        layers (list[int]): List of integers representing the number of units in each layer
        activations (str or list[str], optional): Either a single string or a list of strings with the same length as layers (one for each layer) 
            representing the activation functions. Defaults to "sigmoid".
        optimizer (str, optional): The optimization algorithm used. Defaults to "adam".
        metrics (list, optional): A list of metrics to evaluate the model. Defaults to ['accuracy'].
        loss (str, optional): _description_. Defaults to "binary_crossentropy".

    Returns:
        _type_: _description_
    """
    # create model
    model = Sequential()
    # Add the first layer of the model
    # We will use a simple network with just Dense layers but note that you can combine several type of layers in your model
    if (isinstance(activations, list) and len(activations) == len(layers)):
        activation = activations[0]
        model.add(Dense(units=layers[0], input_shape=(layers[0],), activation=activation))
        for layer, activation in zip(layers[1:], activations[1:]):
            model.add(Dense(units=layer, activation=activation))
    else:
        assert isinstance(activations, str), "activations must be either a single string or a list of strings with the same length as layers (one for each layer)"
        model.add(Dense(units=layers[0], input_shape=(layers[0],), activation=activations))
        for layer in layers[1:]:
            model.add(Dense(units=layer, activation=activations))
    # Compile model
    model.compile(loss=loss, optimizer=optimizer, metrics=metrics)
    return model

### Evaluating the model
Now, it is time to evaluate this model using stratified cross validation in the scikit-learn framework.

Pass the number of training epochs to the KerasClassifier, using reasonable default values. Verbose output is also turned off.

We will use this step as test consisting on training for a short while and evaluate them in order to choose the best model to train for a longer time 

In [28]:
# Create the model, The output layer contains a single neuron in order to make binary predictions. 
# It uses the sigmoid activation function in order to produce a probability output in the range of 0 to 1 that can easily and automatically be converted to crisp class values.
# And we set the input layer to 21 cause there is exactly 21 element in each training examples
model = Create_Sequential(layers=[21, 10, 1], activations=["relu", "relu", "sigmoid"]) # Start with a default simple model (entry -> output)
model.summary()
classifier = KerasClassifier(model=model, epochs=100, batch_size=16, verbose=0)
kfold = StratifiedKFold(n_splits=10, shuffle=True)
print("Training Started")
results = cross_val_score(estimator=classifier, X=Inputs, y=Outputs, cv=kfold)
print(f"Mean accuracy: {round(results.mean()*100, 2)}, Std in accuracy: {round(results.std()*100, 2)}")

Model: "sequential_24"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_71 (Dense)            (None, 21)                462       
                                                                 
 dense_72 (Dense)            (None, 10)                220       
                                                                 
 dense_73 (Dense)            (None, 1)                 11        
                                                                 
Total params: 693
Trainable params: 693
Non-trainable params: 0
_________________________________________________________________
Training Started


INFO:tensorflow:Assets written to: ram://52c01b75667048379a1e2ffa87e3c6ad/assets


INFO:tensorflow:Assets written to: ram://52c01b75667048379a1e2ffa87e3c6ad/assets
2023-05-12 02:49:53.860025: W tensorflow/core/util/tensor_slice_reader.cc:96] Could not open ram://c2df7b5aa3924d3093c8c34e760e7ceb: INVALID_ARGUMENT: ram://c2df7b5aa3924d3093c8c34e760e7ceb is a directory.


INFO:tensorflow:Assets written to: ram://2740ced3f7664320ab7b5f42bd8a4a0b/assets


INFO:tensorflow:Assets written to: ram://2740ced3f7664320ab7b5f42bd8a4a0b/assets
2023-05-12 02:50:12.575637: W tensorflow/core/util/tensor_slice_reader.cc:96] Could not open ram://ef3a3e2c006d4adaba1b730986fc97cd: INVALID_ARGUMENT: ram://ef3a3e2c006d4adaba1b730986fc97cd is a directory.


...

INFO:tensorflow:Assets written to: ram://3be6688162774c309849ef787771754f/assets
2023-05-12 02:53:00.912971: W tensorflow/core/util/tensor_slice_reader.cc:96] Could not open ram://57e44000521740789492f2ca83e77e12: INVALID_ARGUMENT: ram://57e44000521740789492f2ca83e77e12 is a directory.


INFO:tensorflow:Assets written to: ram://5cfc9b25c7264c2eb4eb58f1e9cb61c3/assets


INFO:tensorflow:Assets written to: ram://5cfc9b25c7264c2eb4eb58f1e9cb61c3/assets


2023-05-12 02:53:25.499217: W tensorflow/core/util/tensor_slice_reader.cc:96] Could not open ram://fd07643b05a14abf9e755f6fe94bef64: INVALID_ARGUMENT: ram://fd07643b05a14abf9e755f6fe94bef64 is a directory.


Mean accuracy: 87.16, Std in accuracy: 1.48


### Finally Train the best model

After these steps it's likely that we found a model (Topology, activations, loss, optimizer). So let's finally train this model to get better results

In [29]:
model.fit(x=Inputs, y=Outputs, batch_size=16, epochs=1000, shuffle=True) # This will train the model for 1000 epochs. Each epoch train with the whole dataset

Epoch 1/1000
371/371 [==============================] - 1s 633us/step - loss: 16.0134 - accuracy: 0.6792
Epoch 2/1000
371/371 [==============================] - 0s 646us/step - loss: 3.1214 - accuracy: 0.7637
Epoch 3/1000
371/371 [==============================] - 0s 639us/step - loss: 2.7433 - accuracy: 0.7647
Epoch 4/1000
371/371 [==============================] - 0s 637us/step - loss: 1.8805 - accuracy: 0.7705
Epoch 5/1000
371/371 [==============================] - 0s 765us/step - loss: 1.7186 - accuracy: 0.7855
Epoch 6/1000
371/371 [==============================] - 0s 662us/step - loss: 2.5462 - accuracy: 0.7693
Epoch 7/1000
371/371 [==============================] - 0s 655us/step - loss: 1.9390 - accuracy: 0.7889
Epoch 8/1000
371/371 [==============================] - 0s 661us/step - loss: 1.4542 - accuracy: 0.7956
Epoch 9/1000
371/371 [==============================] - 0s 656us/step - loss: 1.5705 - accuracy: 0.7884
Epoch 10/1000
371/371 [==============================] - 0s 649

### Use our model to make predictions

Now we can use our trained model to make predictions using the predict method

In [25]:
# Prediction data must be and ndarray, list of ndarray or list of 
model.predict("Prediction data on here")

1/1 [==============================] - 0s 49ms/step


array([[1.]], dtype=float32)

### Save and Load a model

Now we can save our model in order to use it for later training or prediction

In [30]:
model.save("../../Models/mymy.log") # This will save the model in the Models/mymy.log folder
                                 # For later utilisation just load the model from the directory using tf.keras.models.load_model function

INFO:tensorflow:Assets written to: ../Models/mymy.log/assets


INFO:tensorflow:Assets written to: ../Models/mymy.log/assets
